# Example of forward modeling centerline data with Meanderpy

### Import Packages

In [1]:
import sys
sys.path.append('/Users/cole/Dropbox/software/python/')
import channelmapper as cm
import meanderpy as mp

In [2]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import importlib

In [3]:
importlib.reload(mp); # Reimport meanderpy if changes are made to .py

In [4]:
%matplotlib qt

### Import Centerline Data

In [5]:
dirname = '/Users/cole/Dropbox/TrinityRiver_Project/cutoff_analysis/data/final_centerline_pts_shapefiles/smoothed_centerline_points/'

In [6]:
# Get list of all files in a given directory sorted by name
cl_list = [f for f in os.listdir(dirname) if f.endswith('.csv')]
cl_list.sort(key = lambda x: x.split('_',)[5])
print(cl_list)

['smoothed_centerline_pts_LT05_025039_19841121.csv', 'smoothed_centerline_pts_LT05_025039_19851007.csv', 'smoothed_centerline_pts_LT05_025039_19861026.csv', 'smoothed_centerline_pts_LT05_025039_19871029.csv', 'smoothed_centerline_pts_LT05_025039_19881218.csv', 'smoothed_centerline_pts_LT05_025039_19891205.csv', 'smoothed_centerline_pts_LT05_025039_19901106.csv', 'smoothed_centerline_pts_LT05_025039_19910330.csv', 'smoothed_centerline_pts_LT05_025039_19921026.csv', 'smoothed_centerline_pts_LT05_025039_19930927.csv', 'smoothed_centerline_pts_LT05_025039_19940407.csv', 'smoothed_centerline_pts_LT05_025039_19950221.csv', 'smoothed_centerline_pts_LT05_025039_19960224.csv', 'smoothed_centerline_pts_LT05_025039_19970821.csv', 'smoothed_centerline_pts_LT05_025039_19980504.csv', 'smoothed_centerline_pts_LT05_025039_19990827.csv', 'smoothed_centerline_pts_LT05_025039_20000203.csv', 'smoothed_centerline_pts_LT05_025039_20011120.csv', 'smoothed_centerline_pts_LT05_025039_20021123.csv', 'smoothed_c

### Read in the first centerline (1984) 

In [7]:
cl_1984 = pd.read_csv(dirname+cl_list[0]) #centerline in csv format

In [8]:
#convert to numpy array
centerline1984 = cl_1984.values
x1984 = centerline1984[:,0]
y1984 = centerline1984[:,1]
z1984 = np.zeros(len(y1984),float)

In [9]:
#plot for inspection
fig = plt.figure()
plt.plot(x1984,y1984,'darkblue', label='1984')
plt.axis('equal')
plt.title('Trinity River Centerline - 1984');

## Forward modeling

### Meanderpy parameterization

In [10]:
W = 100                      # Estimate
D = (W/18.8)**0.7092
pad = 100                    # padding (number of nodepoints along centerline)
deltas = 25                  # sampling distance along centerline

dt_year = 365*24*60*60.0     # time step (s); equal to 1 year
nit_years = 36               # number of iterations; dt is one year, 1984-2020 = 36 years/iterations        

crdist = 1.5*W               # threshold distance at which cutoffs occur; Rule of thumb ~1.5W
kl = 30/(365*24*60*60.0)     # migration rate constant (m/s)
kv =  1.0E-12                # vertical slope-dependent erosion rate constant (m/s)
dens = 1000                  # density of water (kg/m3)

saved_ts_years = 1           # which time steps will be saved; save the channel position every 12 months

S1 = 0.0                     #channel bed slope

#depths = D * np.ones((nit_years,)) + np.repeat(1.5*(np.random.random_sample(int(nit_years/saved_ts_years))-0.5), saved_ts_years)
depths = D * np.ones((nit_years,))  # depth array

Cfs = 0.005 * np.ones((nit_years,)) # dimensionless Chezy friction factor array

                             # these don't affect the plan-view migration pattern
t1 = 1                       # time step when incision starts
t2 = 1                       # time step when lateral migration starts
t3 = 100                     # time step when aggradation starts
aggr_factor = 4e-9           # aggradation factor (m/s, about 0.18 m/year, it kicks in after t3)

### Build 'channel' and 'channelbelt' objects with the centerline

In [11]:
ch1984 = mp.Channel(x1984,y1984,z1984,W,D)
chb1984 = mp.ChannelBelt(channels=[ch1984],cutoffs=[],cl_times=[0.0],cutoff_times=[])

In [12]:
fig = chb1984.plot('strat', 20, 60, chb1984.cl_times[-1], len(chb1984.channels)) # plotting

### Migrate the channel using the above parameterization

In [13]:
chb1984.migrate(nit_years,saved_ts_years,deltas,pad,crdist,depths,Cfs,kl,kv,dt_year,dens,t1,t2,t3,aggr_factor)

100%|██████████| 36/36 [00:47<00:00,  1.33s/it]


In [14]:
fig = chb1984.plot('strat', 20, 60, chb1984.cl_times[-1], len(chb1984.channels)) # plotting

### Look at the first and last channel centerline locations

In [15]:
modeled_2020_centerline = chb1984.channels[-1]

In [16]:
fig = plt.figure()
plt.plot(x1984,y1984,'lightgrey', label = 'Actual 1984 Centerline')
plt.plot(modeled_2020_centerline.x,modeled_2020_centerline.y,'k--', label ='Modeled 2020 Centerline')

plt.axis('equal')
plt.legend();